# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [9]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [10]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,実行,じっこう
1,貢献,こうけん
2,除く,のぞく
3,トップ,トップ
4,僅か,わずか


#### load Models

In [11]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [12]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [13]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題5

つぎのことばの使い方として最もよいものを、1・2・3・4から一つえらびなさい。

31. 内容
	1.	修理のため、エアコンの内容を一度取り出します
	2.	鍋の中にカレーの内容を入れて、1時間くらい煮てください
	3.	古い財布から新しい財布へ内容を移しました
	4.	この手紙の内容は、ほかの人には秘密にしてください

32. 活動
	1.	彼は有名なロック歌手だったが、今は活動していない
	2.	山に登ると、新鮮な空気が活動していて気持ちがいい
	3.	さっきまで活動していたパソコンが、急に動かなくなった
	4.	駅前のコンビニは24時間活動しているので便利だ

33. 落ち着く
	1.	この辺りは、冬になると雪が落ち着いて、春になるまで溶けません
	2.	シャツにしみが落ち着いてしまって、洗ってもきれいになりません
	3.	あそこの木の上に美しい鳥が落ち着いています
	4.	大好きなこの曲を聞くと、いつも気持ちが落ち着きます

34. ぐっすり
	1.	遠慮しないで、ぐっすり食べてください
	2.	優勝できたのは、毎日ぐっすり練習したからだと思う
	3.	今日は疲れているので、朝までぐっすり眠れそうだ
	4.	古い友人と久しぶりに会って、ぐっすりおしゃべりした

35. 性格
	1.	日本の古い性格に興味があるので、神社やお寺によく行きます
	2.	森さんはおとなしい性格で、自分の意見はあまり言いません
	3.	値段が高くても、塗装で性格のいい車を買うつもりです
	4.	音楽の性格を伸ばすために、5歳から専門家の指導を受けました
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"""You are a Japanese teacher. Your job is to write 5 word usage for candidates from the following sentences, select the most appropriate usage of the word from options 1, 2, 3, and 4. (Includes Japanese idiomatic expressions and fixed collocations.)
"""
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            実行    じっこう
1            貢献    こうけん
2            除く     のぞく
3           トップ     トップ
4            僅か     わずか
...         ...     ...
2134        うなる     うなる
2135          脳      のう
2136         臨む     のぞむ
2137         機嫌     きげん
2138        駆ける     かける

[2139 rows x 2 columns]


In [14]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [15]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で商品の特徴や価格を尋ねる | 電車やバスの乗り方を尋ねる | 今日の天気や気温について話す| 映画やドラマの感想を話す | 仕事のスケジュールや業務内容を話す"})
display(Markdown(kanji["question"]))

# 日本語語彙使用問題

## 問題5

つぎのことばの使い方として最もよいものを、1・2・3・4から一つえらびなさい。

### 31. 実行
1. 明日の会議で新しいプロジェクトの実行について話し合います
2. この薬は胃の実行を良くする効果があります
3. 彼は約束の実行をして、友達を助けました
4. 会社の実行が悪くなって、多くの社員が辞めました

**正解：1**

**解説**：「実行」は計画やアイデアを実際に行うことを意味します。この問題は「実行」の正しい使い方を理解しているかを確認するものです。「プロジェクトの実行」は正しい使い方ですが、胃の機能、約束を守ること、会社の状況には使いません。

### 32. 除く
1. 彼は家族を除くすべての人に秘密を話しました
2. 窓を開けて、部屋の空気を除いてください
3. 彼女は毎朝、家の前の雪を除きます
4. 彼は友達の悩みを除くために、一生懸命話を聞きました

**正解：1**

**解説**：「除く」は「取り除く」「含めない」という意味です。この問題は前置詞的な使い方と動詞としての使い方の違いを理解しているかを確認します。「〜を除く」で「〜以外の」という意味で使うのが正しいです。2と3では「取り除く」という意味で使えますが、表現が不自然です。4は「取り除く」対象として「悩み」は使えますが、「除く」より「解消する」などが自然です。

### 33. 機嫌
1. 子供は機嫌が良くて、ずっと笑っています
2. 上司の機嫌を取るために、いつも彼のコーヒーを入れています
3. この店の機嫌はとても良くて、料理がおいしいです
4. 新しい靴を買ったら、足の機嫌が良くなりました

**正解：2**

**解説**：「機嫌」は人の気分や心の状態を表す言葉です。この問題は「機嫌」が人の感情状態に使われることを理解しているか確認します。「機嫌を取る」は「相手の気分を良くするために行動する」という意味の慣用表現です。1も正しい使い方ですが、2が最も自然な慣用表現です。3と4は店や足に「機嫌」を使うのは不適切です。

### 34. トップ
1. 彼はクラスのトップで、いつも試験で一番です
2. この山のトップまで登ると、素晴らしい景色が見えます
3. トップの声で歌うと、のどが痛くなります
4. このケーキのトップにイチゴを置いて飾りました

**正解：1**

**解説**：「トップ」は「最上位」「先頭」「頂点」などを意味します。この問題は「トップ」の様々な使い方の中から最も自然なものを選ぶ問題です。1の「クラスのトップ」は「成績が最も良い」という意味で正しいです。2は「頂上」、4は「